In [1]:
import os
import ijson
import json



In [10]:
# count a NID data file
def count_NID_onefile(filename, require_d3, sr_li):
    result={}
    numo3=0
    numo7=0
    numo3o7=0
    numo5=0
    numo9=0
    numo5o9=0
    polynumber=0
    triangnumber=0
    involnumber=0
    datasmoothT=0
    datasmoothN=0
    datasmoothF=0
    numno_oplane=0
    hodgesplit={}
    invol_divisor_type={}
    polyalreadyin={}
    keyout=[]
    with open(filename, 'r') as p:
        datas=ijson.items(p, 'item')
        for data in datas:
            if data=={} or data==[]:
                continue
            try:
                if data["invol_triple"]=="[]":
                    continue
            except:
                print(filename)
            invol=eval(data["INVOL"])

            if sr_li:
                invols=eval(data["invol_sr_li_tr"])
                if invol not in invols:
                    continue
            if require_d3:
                if data["OPLANES"]["tadpole_cancel"]!=True:
                    continue        

            if str(data["polyid"]) in polyalreadyin.keys():

                if data["SR_list"] not in polyalreadyin[str(data["polyid"])]:
                    polyalreadyin[str(data["polyid"])].append(data["SR_list"])
                    triangnumber+=1
            else:
                if polyalreadyin=={}:
                    polyalreadyin.setdefault(str(data["polyid"]), [data["SR_list"]])
                    polynumber+=1  
                    triangnumber+=1
                else:
                    print("One polyfile has two different polytope")
            if "[h12p, h12n, smooth]" in data["OPLANES"].keys():
                if "True" in data["OPLANES"]["[h12p, h12n, smooth]"]:
                    datasmoothT+=1
                elif "None" in data["OPLANES"]["[h12p, h12n, smooth]"]:
                    datasmoothN+=1
                else:
                    datasmoothF+=1
            involnumber+=1
            if data["OPLANES"]=="free_action":
                print(data["label"], "free_action")
            else:
                if "O3" in data["OPLANES"].keys():
                    if "O5" in data["OPLANES"].keys() or "O9" in data["OPLANES"].keys():
                        print("problem data")
                    if "O7" in data["OPLANES"].keys():
                        numo3o7+=1
                    else:
                        numo3+=1
                elif "O7" in data["OPLANES"].keys():
                    if "O5" in data["OPLANES"].keys() or "O9" in data["OPLANES"].keys():
                        print("problem data")
                    numo7+=1
                elif "O5" in data["OPLANES"].keys():
                    if "O9" in data["OPLANES"].keys():
                        numo5o9+=1
                    else:
                        numo5+=1
                    if require_d3:
                        print(data["OPLANES"])
                        continue
                elif "O9" in data["OPLANES"].keys():
                    numo9+=1
                elif "No Oplane" in data["OPLANES"].keys() or data["OPLANES"]=={}:
                    numno_oplane+=1
                    #print("No oplane",data["OPLANES"], filename, data["label"], data["INVOL"])
                else:
                    print("There may be something wrong ",data["OPLANES"], filename, data["label"], data["INVOL"])
                divisors=eval(data["divisors_hodge"] )
                if str(data["hodge_split_p_n"]) in hodgesplit.keys():
                    hodgesplit[str(data["hodge_split_p_n"])]+=1
                else:
                    hodgesplit.setdefault(str(data["hodge_split_p_n"]), 1)
                type_invols_divisor=[]
            # rigid 1, willson 2, exact 2.1, k3 3, sd1 4, sd2 5

            divisors=eval(data["divisors_hodge"].replace("'", ""))
            type_invols_divisor=[]
            # rigid 1(del Pelzzo 1, non-shrinkable 1.2), willson 2, k3 3, sd1 4, sd2 5
            for invo in invol:
                divisor_index=invo[0]
                divisor=divisors[divisor_index]
                if divisor[1]!=0:
                    if divisor[3]!=0:
                        if divisor[1]==1:
                            type_invols_divisor.append(7)
                        else:
                            type_invols_divisor.append(2)
                    else:
                        type_invols_divisor.append(0)
                        print("有点问题", divisor)
                else:
                    if divisor[2]==0:
                        # 23都为0
                        if divisor[3]<10:
                            type_invols_divisor.append(1)
                        else:
                            type_invols_divisor.append(6)
                    elif divisor==[1,0,1,20]:
                        type_invols_divisor.append(3)
                    elif divisor==[1,0,1,21]:
                        type_invols_divisor.append(4)
                    elif divisor==[1,0,2,30]:
                        type_invols_divisor.append(5)
                    else:
                        type_invols_divisor.append(0)
            type_invols_divisor.sort()
            if str(type_invols_divisor) in invol_divisor_type.keys():
                invol_divisor_type[str(type_invols_divisor)]+=1
            else:
                invol_divisor_type.setdefault(str(type_invols_divisor), 1)
    if keyout!=[]:
        print(keyout, "This not empty, something wrong")
    result.setdefault("h11, polynumber, trinumber", [0, polynumber, triangnumber])
    #print("total polytope: ", polynumber," , total triangulation", triangnumber," 个", numo3, numo7, numo5, numo3o7)        
    result.setdefault("hodge_split", hodgesplit)
    result.setdefault("no_oplane, numo3, numo7, numo3o7, numo5, numo9, numo5o9, involnumber",[numno_oplane, numo3, numo7, numo3o7,numo5,numo9,numo5o9,  involnumber])
    result.setdefault("invol_divisor_type",invol_divisor_type)
    result.setdefault("smooth_N_T_F",[datasmoothN, datasmoothT, datasmoothF])
    return result
    #return [polynumber, triangnumber, numno_oplane, numo3, numo7, numo3o7,numo5,numo9,numo5o9,  involnumber, invol_divisor_type,  hodgesplit, datasmoothN, datasmoothT, datasmoothF]
    

In [11]:
        
# count a REF data file 
def count_ref_onefile(filename, require_d3, ref):
    numo3_ref=0
    numo7_ref=0
    numo3o7_ref=0
    numo5_ref=0
    numo9_ref=0
    numo5o9_ref=0
    numtotal=0
    numnofix=0
    triangnumber=0
    involnumber=0
    datasmoothT=0
    datasmoothN=0
    datasmoothF=0
    invol_divisor_type={}
    #trinum=[]
    SRIS=[]
    result={}
    with open(filename, 'r') as p:
        datas=ijson.items(p, 'item')
        #try:
        for data in datas:
            if data=={} or data==[]:
                continue
            if data["SR_list"] not in SRIS:
                triangnumber+=1
                SRIS.append(data["SR_list"])
            else:
                print("似乎有重复的，",filename,data["SR_list"])
            if "(" in data["divisor"]:
                divisors=[eval(x.split(")")[0]) for x in data["divisor"].split("(")[1:]]
            else:
                divisors=eval(data["divisor"])
            if "invol"+str(ref) not in data.keys():
                print(filename + " no invol"+str(ref))
                continue
            for dataone in data["invol"+str(ref)]:
                if require_d3:
                    if "satisfied" not in dataone:
                        continue
                involnumber+=1
                if "[h12p, h12n, smooth]" in dataone:
                    if "True" in dataone:
                        datasmoothT+=1
                    elif "None" in dataone:
                        datasmoothN+=1
                    else:
                        datasmoothF+=1
                else:
                    if "total" in dataone or "no fix" in dataone:
                        datasmoothN+=1
                        
                if ref in [1,3]:
                    
                    if "O5" in dataone or "O9" in dataone:
                        print("Problem data, something wrong:", dataone, filename)
                    elif "O3" in dataone:
                        if "O7" in dataone:
                            numo3o7_ref+=1
                        else:
                            numo3_ref+=1
                    elif "O7" in dataone:
                        numo7_ref+=1
                    else:
                        if "total" in dataone or "trivial" in dataone:
                            numtotal+=1
                        elif "no fixed" in dataone:
                            numnofix+=1
                        else:
                            print("other type oplane", filename, dataone, data["polyid"], data["tri_id"])
                else:
                    
                    if "O3" in dataone or "O7" in dataone:
                        print("Problem data, something wrong:", dataone, filename)
                    if "O5" in dataone:
                        if "O9" in dataone:
                            numo5o9_ref+=1
                        else:
                            numo5_ref+=1
                    elif "O9" in dataone:
                        numo9_ref+=1
                    else:

                        if "total" in dataone or "trivial" in dataone:
                            numtotal+=1
                        elif "no fixed" in dataone:
                            numnofix+=1
                        else:
                            print("other type oplane", filename, dataone, data["polyid"], data["tri_id"])
                type_invols_divisor=[]

                # rigid 1, willson 2, exact 2.1, k3 3, sd1 4, sd2 5
                # 0 represent other type divisor
                invols=eval(dataone.split(";")[0])
                for invol in invols:
                    divisor=divisors[invol]
                    if divisor[1]!=0:
                        if divisor[3]!=0:
                            if divisor[1]==1:
                                type_invols_divisor.append(7)
                            else:
                                type_invols_divisor.append(2)
                        else:
                            type_invols_divisor.append(0)
                            print("problem ", divisor)
                    else:
                        if divisor[2]==0:
                            # 23都为0
                            if divisor[3]<10:
                                type_invols_divisor.append(1)
                            else:
                                type_invols_divisor.append(6)
                        elif divisor==[1,0,1,20]:
                            type_invols_divisor.append(3)
                        elif divisor==[1,0,1,21]:
                            type_invols_divisor.append(4)
                        elif divisor==[1,0,2,30]:
                            type_invols_divisor.append(5)
                        else:
                            type_invols_divisor.append(0)
                type_invols_divisor.sort()
                if str(type_invols_divisor) in invol_divisor_type.keys():
                    invol_divisor_type[str(type_invols_divisor)]+=1
                else:
                    invol_divisor_type.setdefault(str(type_invols_divisor), 1)
            if not require_d3:
                if sum([numo3_ref, numo7_ref, numo3o7_ref, numo5_ref, numo9_ref,numo5o9_ref,numtotal, numnofix])!=involnumber:
                    print("something wrong,number not meet ",filename, sum([numo3_ref, numo7_ref, numo3o7_ref, numo5_ref, numo9_ref,numo5o9_ref,numtotal, numnofix]), involnumber)
    result.setdefault("h11, polynumber, triangnumber", [0, 1, triangnumber])
    result.setdefault("numo3_ref, numo7_ref, numo3o7_ref, numo5_ref, numo9_ref, numo5o9_ref, involnumber",[numo3_ref, numo7_ref, numo3o7_ref, numo5_ref, numo9_ref, numo5o9_ref, involnumber])
    result.setdefault("invol_divisor_type",invol_divisor_type)
    result.setdefault("[total_fixed, not fix]", [numtotal, numnofix])
    result.setdefault("smooth_N_T_F",[datasmoothN, datasmoothT, datasmoothF])
    return result
    #return [numo3_ref, numo7_ref, numo3o7_ref,numo5_ref,numo9_ref,numo5o9_ref, numtotal, numnofix, 1, triangnumber, involnumber, invol_divisor_type,  datasmoothN,datasmoothT,datasmoothF]



In [4]:
# count a file of data file
def count_REF_many_files(h11, filepath, required3, ref):
    result={}
    fileori=os.listdir(filepath)
    numo3_ref=0
    numo7_ref=0
    numo3o7_ref=0
    numo5_ref=0
    numo9_ref=0
    numo5o9_ref=0
    numtotal_ref=0
    notfix_ref=0
    polynumber=0
    triangnumber=0
    involnumber=0
    datasmoothT=0
    datasmoothN=0
    datasmoothF=0
    invol_divisor_type={}
    #listmany=[numo3_ref, numo7_ref, numo3o7_ref, numo5_ref, numo9_ref,numo5o9_ref, numtotal_ref,notfix_ref,polynumber, triangnumber, involnumber, invol_divisor_type, datasmoothN,datasmoothT,datasmoothF]
    result.setdefault("h11, polynumber, triangnumber", [h11, polynumber, triangnumber])
    result.setdefault("numo3_ref, numo7_ref, numo3o7_ref, numo5_ref, numo9_ref, numo5o9_ref, involnumber",[numo3_ref, numo7_ref, numo3o7_ref, numo5_ref, numo9_ref, numo5o9_ref, involnumber])
    result.setdefault("invol_divisor_type",invol_divisor_type)
    result.setdefault("[total_fixed, not fix]", [numtotal_ref, notfix_ref])
    result.setdefault("smooth_N_T_F",[datasmoothN, datasmoothT, datasmoothF])



    for file in fileori:
        try:
            resultonefile=count_ref_onefile(filepath+file, required3, ref)
        except:
            print(file)
            continue
        for key in result.keys():
            if type(result[key])==list:
                for i in range(len(result[key])):
                    result[key][i]+=resultonefile[key][i]
            elif type(result[key])==dict:
                for kkey in resultonefile[key].keys():
                    if kkey in result[key].keys():
                        result[key][kkey]+=resultonefile[key][kkey]
                    else:
                        result[key].setdefault(kkey, resultonefile[key][kkey])
            elif type(result[key])==int:
                result[key]+=resultonefile[key]
            else:
                print("这个key是什么类型的，不太清楚")
    return result




# h11<=7, ref, (ALL of this count could take a lot minites.)

In [5]:
# count the result of REF
resultsout=[]
required3=False
ref=3
for h11 in [2,3,4,5,6,7]:
    print(h11)
    filepathREF="../../1<h11<=7/REF/h11="+str(h11)+"/"
    if h11==7:
        filepathREF+="ref"+str(ref)+"/"
    resultsout.append(count_REF_many_files(h11, filepathREF, required3, ref))
with open("../result/REFresultsout_h2-7_required3"+str(required3)+"_ref"+str(ref)+".txt", 'w') as p_out:
    json.dump(resultsout, p_out, indent=2)


2
3
4
5
6
7


In [6]:
# count the result of REF
resultsout=[]
required3=False
ref=1
for h11 in [2,3,4,5,6,7]:
    print(h11)
    filepathREF="../../1<h11<=7/REF/h11="+str(h11)+"/"
    if h11==7:
        filepathREF+="ref"+str(ref)+"/"
    resultsout.append(count_REF_many_files(h11, filepathREF, required3, ref))
with open("../result/REFresultsout_h2-7_required3"+str(required3)+"_ref"+str(ref)+".txt", 'w') as p_out:
    json.dump(resultsout, p_out, indent=2)

2
3
4
5
6
7


In [7]:
# count the result of REF
resultsout=[]
required3=False
ref=2
for h11 in [2,3,4,5,6,7]:
    print(h11)
    filepathREF="../../1<h11<=7/REF/h11="+str(h11)+"/"
    if h11==7:
        filepathREF+="ref"+str(ref)+"/"
    resultsout.append(count_REF_many_files(h11, filepathREF, required3, ref))
with open("../result/REFresultsout_h2-7_required3"+str(required3)+"_ref"+str(ref)+".txt", 'w') as p_out:
    json.dump(resultsout, p_out, indent=2)

2
3
4
5
6
7


# h11>7, REF

In [12]:
ref=3
required3=False
resultsout=[]
for h11 in [8,9,10,11,12]:
    print(h11)
    filepathREF="../../8<=h11<=12/REF/h11="+str(h11)+"/"
    resultsout.append(count_REF_many_files(h11, filepathREF, required3, ref))
with open("../result/REFresultsout_h8-12_required3"+str(required3)+"_ref"+str(ref)+".txt", 'w') as p_out:
    json.dump(resultsout, p_out, indent=2)


8
9
10
11
12
